# **Relatório trabalho 1 AED III**
## **Algoritmos exatos**
Foram feitos testes com 3 algoritmos exatos: Força Bruta, Branch and Bound, e
Held Karp; dentre eles, o algoritmo de Held Karp foi o que mais se destacou devido
sua velocidade de resolução, entretanto não foi possível resolver instâncias como
TSP4 e TSP5, devido sua complexidade de espaço já os algoritmos de Força Bruta
e Branch and Bound (que têm complexidade temporal $O(n!)$) não conseguiram
resolver o TSP3 em tempo útil.
### **Held Karp**
É um algoritmo de programação dinâmica, isto é, um método que busca encontrar a solução de subproblemas para obter a solução ótima geral. Temos como entrada a matriz de adjacências e o objetivo de encontrar o ciclo hamiltoniano, o algoritmo começa com a escolha de uma cidade inicial qualquer, e calcula o custo para cada possibilidade de conjunto de cidades intermediárias, onde os caminhos são armazenados para evitar recálculos e rastrear o caminho mais curto.
### **Complexidade e Resultados**
Held Karp tem complexidade temporal $Θ(2^n*n^2)$, no entanto a complexidade de
espaço é $Θ(n*2^n)$. Com ele tivemos ótimos resultados para as três primeiras
instâncias TSP1,TSP2,TSP3, conseguindo executar todos em menos de 1s, o que é
discrepante dos resultados que os outros algoritmos tiveram.

| *Algoritmos*       | *TSP1*  | *TSP2* | *TSP3* |
|--------------------|---------|--------|--------|
| *Brute Force*      | 117.65s | 0.002s | 8h+    |
| *Branch and Bound* | 12.83s  | 0.002s | 8h+    |
| *Held Karp*        | 0.013s  | 0s     | 0.04s  |


Já as instâncias maiores TSP4 e TSP5 tiveram grandes problemas de desempenho
devido a falta de memória, logo não seria possível obter um resultado exato pela
falta de recursos, então fizemos algumas estimativas:

**Estimativas de Memória &rarr; $Θ(n*2^n)$**

TSP5: Temos $29 ∗ 2^{29}$ , o que nos dá um valor próximo a $15 ∗ 10^9$ unidades de
memória, que multiplicado por 4 bytes (inteiro em Python) temos um valor próximo a $60 ∗ 10^9$, um resultado próximo a 60 GB;

TSP4: Temos $44 ∗ 2^{44}$, que nos leva a $774,056 × 10^{12}$, multiplicado por 4 temos
$3.096,224 ∗ 10^{12}$com a conversão temos um valor próximo de 3,1 PB

**Estimativas de Tempo &rarr; $Θ(2^n*n^2)$**

Considerando um processador com velocidade média de 4 GHz:

TSP5: Temos $2^{29} ∗ 29^2$, que da $451, 508 × 10^9$ instruções, dividido por $4 ∗ 10^9$, que é a velocidade média dos ciclos do processador por segundos, temos 113s

TSP4: Temos $2^{44} ∗ 44^2$, que da $3, 40584722 × 10^{16}$, com a divisão temos $34.0584722 ∗ 10^7 /4$, que é algo próximo a 85146180 segundos, aproximadamente 2 anos e 7 messes.

OBS: Estas estimativas consideram apenas valores brutos, não levando em conta fatores como escalabilidade do sistema de execução, dentre outros fatores externos, logo o resultado real pode ser diferente.

Abaixo um exemplo para o TSP1 do algoritmo de Held Karp

In [ ]:
import itertools

def held_karp(dists):
    n = len(dists)
    C = {}
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            for k in subset:
                prev = bits & ~(1 << k)

                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                C[(bits, k)] = min(res)
    bits = (2**n - 1) - 1
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, _ = min(res)

    return opt

graph = [
[0   ,29  ,20  ,21  ,16  ,31  ,100 ,12  ,4   ,31  ,18],
[29  ,0   ,15  ,29  ,28  ,40  ,72  ,21  ,29  ,41  ,12],
[20  ,15  ,0   ,15  ,14  ,25  ,81  ,9   ,23  ,27  ,13],
[21  ,29  ,15  ,0   ,4   ,12  ,92  ,12  ,25  ,13  ,25],
[16  ,28  ,14  ,4   ,0   ,16  ,94  ,9   ,20  ,16  ,22],
[31  ,40  ,25  ,12  ,16  ,0   ,95  ,24  ,36  ,3   ,37],
[100 ,72  ,81  ,92  ,94  ,95  ,0   ,90  ,101 ,99  ,84],
[12  ,21  ,9   ,12  ,9   ,24  ,90  ,0   ,15  ,25  ,13],
[4   ,29  ,23  ,25  ,20  ,36  ,101 ,15  ,0   ,35  ,18],
[31  ,41  ,27  ,13  ,16  ,3   ,99  ,25  ,35  ,0   ,38],
[18  ,12  ,13  ,25  ,22  ,37  ,84  ,13  ,18  ,38  ,0]
]

minCost = held_karp(graph)
print(f'Melhor custo encontrado: {minCost}')


Melhor custo encontrado: 253


##**Algoritmos Aproximativos**
Para chegar em resultados aproximados usamos um algoritmo genético com 3 padrões diferentes de parâmetros, em teoria, um melhor que o outro. Um algoritmo genético é um algoritmo que simula a seleção natural usando conceitos de gerações, população, cruzamento e mutação. Cada indivíduo será representado por um ciclo hamiltoniano onde cada vértice é um gene. O custo do ciclo será o fator determinante de aptidão, quanto menor, melhor o indivíduo.

Em questão de complexidade, um algoritmo genético não se baseia apenas pelo tamanho da entrada. Deve ser levado em conta também os parâmetros passados, no nosso caso, estamos passando o numero de gerações, quantidade de indivíduos por geração, taxa de cruzamentos e taxa de mutações. Algoritmos genéticos podem ter também um critério de parada, que pode afetar sua complexidade. Para este trabalho o algoritmo genético tem apenas como "critério de parada" o numero de gerações.

###**Parâmetros**
####**Cruzamento**
O cruzamento é a parte mais critica do algoritmo genético pois é o responsável pela criação de novos indivíduos. Existem diversos métodos para diferentes para diferentes problemas. Para este trabalho usaremos um método mais simples, o *cruzamento de ponto único* com uma taxa de 50%. Este método apenas divide dois indivíduos em duas partes e usa uma parte de cada para formar um novo individuo.

####**Mutação**
A mutação em um algoritmo genético nos da a variabilidade genética, nos permitindo explorar indivíduos diferentes dos antepassados. Assim como o cruzamento, a mutação também possui diferentes métodos para diferentes problemas. Nesse trabalho usaremos a *mutação de troca* com uma taxa de 5%. Este método seleciona aleatoriamente dois genes do individuo e os invertem de posição.

#### **Número de gerações**
O número de gerações é basicamente a quantidade de vezes em que geraremos uma nova população. A primeira é gerada aleatoriamente, enquanto as futuras gerações serão filhos da geração passada. A quantidade de gerações é um fator de extrema importância, é necessário encontrar um equilíbrio a depender do problema e dos limites de tempo e espaço. Poucas gerações podem(ou não)nos dar uma resposta longe da ótima, enquanto muitas gerações podem levar muito tempo e não gerar uma melhora significativa.

#### **População**
A população é o números de indivíduos por geração. Este parâmetro nos da uma amostra mais diversificada do espaço de busca, melhorando a exploração de soluções boas. Assim como o número de gerações, um valor alto pode nos custar caro sem melhoras significativas e valores baixos podem nos dar respostas insatisfatórias, com isso, é importante também encontra um equilíbrio a depender do problema.

###**Parâmetros para nossos testes**
Para os testes de qualidade da resposta e de tempo utilizaremos 3 diferentes entradas para os parâmetros do algoritmo genético, que serão os seguintes:

|             | *Entrada A* | *Entrada B* | *Entrada C* |
|-------------|-------------|-------------|-------------|
| *População* | 100         | 200         | 500         |
| *Gerações*  | 1000        | 3000        | 8000        |


A taxa de cruzamento(50%) e a taxa de mutação(5%) se manterão iguais para os testes A, B e C, a unica diferença aqui será no numero de gerações e da quantidade de individuos que será gerado a cada geração nova.

###**Resultados**
Os resultados encontrados para cada teste são os seguintes:

| Arquivo | Custo Ótimo | *Custo A* | *Tempo A* | *Custo B* | *Tempo B* | *Custo C* | *Tempo C* |
|---------|:-----------:|-----------|-----------|-----------|-----------|-----------|-----------|
| TSP1    | 253         | 259       | 0,62s     | 301       | 3,62s     | 267       | 23,03s    |
| TSP2    | 1248        | 1248      | 0,39s     | 1248      | 2,40s      | 1248      | 15,94s    |
| TSP3    | 1194        | 1544      | 0,76s     | 1194      | 4,43s     | 1194      | 28,76s    |
| TSP4    | 7013        | 21278     | 1,87s     | 16613     | 11,54s    | 8945      | 1m 16s    |
| TSP5    | 27603       | 49765     | 1,26s     | 50230     | 7,79s     | 40698     | 51,02s    |

Os parâmetros de entrada A são os que oferecem a menor variabilidade genética em comparação aos outros por conta do numero de menor de gerações e população, o que nos da um tempo de execução melhor, porem, chegou em resultados satisfatórios nos arquivos TSP1 e TSP3, e chegou na resposta ótima no TSP2. Nos arquivos TSP4 e TSP5 acabou ficando bem distante da resposta ótima.

Os parâmetros de entrada B possuem uma variabilidade genética mediana em comparação aos outros, levando um pouco mais de tempo que os parâmetros A, ele chegou nas respostas ótimas no TSP2 e TSP3 e obteve uma resposta satisfatória no TSP1. Para o TSP4 e TSP5 não obteve muita vantagem em relação aos parâmetros A mesmo levando mais tempo.

Os parâmetros de entrada C nos oferecem a maior variabilidade genética entre os parâmetros testados, porem, leva mais tempo de execução. No arquivo TSP1, acabou sendo pior que os parâmetros A, o que nos mostra que com a aleatoriedade nem sempre a diversidade e tempo de execução melhoram os resultados. Nos arquivos TSP2 e TSP3 foi possível chegar na resposta ótima, e no TSP4 obteve uma resposta satisfatória. Assim como os demais parâmetros, o TSP5 ficou um pouco distante.

Abaixo um exemplo para o TSP1 do algoritmo genético





In [ ]:
import random

def firstPopulation(populationSize, numGenes):
    return [random.sample(range(numGenes), numGenes) for _ in range(populationSize)]

def calculateFitness(individual, matrix):
    totalFitness = 0
    for i in range(len(individual) - 1):
        totalFitness += matrix[individual[i]][individual[i + 1]]
    totalFitness += matrix[individual[-1]][individual[0]]
    return 1 / totalFitness

def crossover(firstParent, secondParent):
    crossoverPoint = random.randint(0, len(firstParent) - 1)
    child = firstParent[:crossoverPoint] + [gene for gene in secondParent if gene not in firstParent[:crossoverPoint]]
    return child

def mutate(individual):
    index1, index2 = random.sample(range(len(individual)), 2)
    individual[index1], individual[index2] = individual[index2], individual[index1]
    return individual

def geneticAlgorithm(matrix, populationSize, crossoverRate, mutationRate, maxGenerations) :

    numGenes = 11

    population = firstPopulation(populationSize, numGenes)

    for _ in range(maxGenerations):
        fitnessScore = [calculateFitness(individual, matrix) for individual in population]

        parents = random.choices(population,weights=fitnessScore, k=populationSize)

        newPopulation = []
        for i in range(0, populationSize, 2):
            if random.random() < crossoverRate:
                child1 = crossover(parents[i], parents[i + 1])
                child2 = crossover(parents[i + 1], parents[i ])
                newPopulation.extend([child1, child2])
            else:
                newPopulation.extend([parents[i], parents[i + 1]])

        for i in range(populationSize):
            if random.random() < mutationRate:
                newPopulation[i] = mutate(newPopulation[i])

        population = newPopulation

    bestIndividual = max(population, key=lambda x: calculateFitness(x, matrix))
    bestFitness = 1 / calculateFitness(bestIndividual, matrix)

    return bestFitness

graph = [
[0   ,29  ,20  ,21  ,16  ,31  ,100 ,12  ,4   ,31  ,18],
[29  ,0   ,15  ,29  ,28  ,40  ,72  ,21  ,29  ,41  ,12],
[20  ,15  ,0   ,15  ,14  ,25  ,81  ,9   ,23  ,27  ,13],
[21  ,29  ,15  ,0   ,4   ,12  ,92  ,12  ,25  ,13  ,25],
[16  ,28  ,14  ,4   ,0   ,16  ,94  ,9   ,20  ,16  ,22],
[31  ,40  ,25  ,12  ,16  ,0   ,95  ,24  ,36  ,3   ,37],
[100 ,72  ,81  ,92  ,94  ,95  ,0   ,90  ,101 ,99  ,84],
[12  ,21  ,9   ,12  ,9   ,24  ,90  ,0   ,15  ,25  ,13],
[4   ,29  ,23  ,25  ,20  ,36  ,101 ,15  ,0   ,35  ,18],
[31  ,41  ,27  ,13  ,16  ,3   ,99  ,25  ,35  ,0   ,38],
[18  ,12  ,13  ,25  ,22  ,37  ,84  ,13  ,18  ,38  ,0]
]

bestFitness = geneticAlgorithm(graph, 100, 0.5, 0.05, 1000)

print(f'Melhor custo encontrado: {bestFitness}')

Melhor custo encontrado: 291.0


###**Conclusão**


Algoritmos exatos demonstram alta eficiência na resolução de instancias pequenas, com destaque para o algoritmo de Held Karp, que se destacou na velocidade de resposta nos arquivos TSP1, TSP2 e TSP3. Porem, ao enfrentar instâncias maiores acabou tendo limitações significativas devido a complexidade de tempo e espaço.

Entretanto, os algoritmos aproximativos ofereceram uma solução satisfatória com o tempo de execução baixo em diversos casos, e até obteve a resposta ótima em instâncias pequenas. Seus resultados demonstram a importância de se escolher bem os parâmetros dependendo do problema. Em alguns casos, parâmetros maiores não tiveram melhora significativa na resposta mesmo tendo um tempo de execução a mais considerável. Porem, em instâncias maiores, usar parâmetros maiores nos levou a respostas mais próximas da ótima, mas exigiu mais tempo de execução.

Em paralelo com o dilema *P vs NP*, nossas analises destacam as limitações enfrentadas ao lidar com o TSP, que é um problema *NP-Difícil* e a necessidade de adaptações na busca pela soluções. A compreensão das limitações demonstra ser crucial ao enfrentar essa classe de problemas, onde tivemos que, em alguns casos, abrir mão da resposta ótima se quisermos obter uma resposta em um tempo aceitável.
